In [ ]:
import openai
import glob

client = openai.OpenAI()

batch_files = glob.glob("batch_files/batch_requests_*.jsonl")
batch_ids = []

for batch_file in batch_files:
    with open(batch_file, "rb") as f:
        batch_input_file = client.files.create(file=f, purpose="batch")
        batch_input_file_id = batch_input_file.id
        batch = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={"description": "nightly eval job"}
        )
        batch_ids.append(batch.id)

Batch(id='batch_6826405bece881909c49c666d3bc75e0', completion_window='24h', created_at=1747337307, endpoint='/v1/chat/completions', input_file_id='file-33z1SfQtPjEnwcYqRpjdaz', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1747423707, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
# Retrieve all batch statuses
for batch_id in batch_ids:
    batch = client.batches.retrieve(batch_id)
    print(f"Batch ID: {batch.id}, Created: {batch.created_at}, Status: {batch.status}")

Batch(id='batch_6826405bece881909c49c666d3bc75e0', completion_window='24h', created_at=1747337307, endpoint='/v1/chat/completions', input_file_id='file-33z1SfQtPjEnwcYqRpjdaz', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1747337613, error_file_id=None, errors=None, expired_at=None, expires_at=1747423707, failed_at=None, finalizing_at=1747337610, in_progress_at=1747337310, metadata={'description': 'nightly eval job'}, output_file_id='file-Pikj8giM9fbMYEhrnvcCVs', request_counts=BatchRequestCounts(completed=20, failed=0, total=20))


In [ ]:
# Download all batch outputs
for batch_id in batch_ids:
    batch = client.batches.retrieve(batch_id)
    file_response = client.files.content(batch.output_file_id)
    output_file = f"batch_output_{batch_id}.jsonl"
    with open(output_file, "w") as f:
        f.write(file_response.text)